In [38]:
!pip install datasketch
!pip install tqdm

In [40]:
import nltk
from nltk.util import bigrams
from datasketch import MinHash, MinHashLSH
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
from tqdm import tqdm
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tanuj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [48]:
 df = pd.read_csv("only_telugu_content.v4.2.csv")
 sentences = np.array(df['content'])
 print(sentences)


['లంచగొండి అధికారులు, రాజకీయ నాయకులు అందరూ దొంగలే. కేవలం కూలీలను పట్టుకొని కేసులు పెడతారు.'
 '❤ నువ్వు సూపర్ తల్లి. జై ఉషా జైజై బీఎస్పీ.'
 'అయిన ఈయన కాదు కదా ఎంఎల్ఏ అభ్యర్థి.' ...
 'హైదరాబాద్\u200c: కాగా కిరణ్\u200c తండ్రి, ప్రముఖ రచయిత అల్లం రాజ య్య శుక్రవారం రాత్రే హైదరాబాద్\u200c నుంచి అమెరికాకు పయనమ య్యారు. భారత కాలమానం ప్రకారం ఆదివారం ఉదయానికి ఆయన లూసియానా స్టేట్\u200c వర్సిటీ చేరుకోవచ్చు. ఆ తర్వాతే కిరణ్\u200c మృతదేహాన్ని భారత్\u200c తీసుకువెళ్లాలా? లేదా? తదితర అంశాలపై నిర్ణయం తీసుకునే అవకాశం ఉంది. చంద్రశేఖర్\u200cరెడ్డి మృతదేహాన్ని మాత్రం భారత్\u200cకు రప్పించాలని ఆయన కుటుంబ సభ్యులు కోరు కుంటున్నారు.'
 'నువ్వు నీ పార్టీలోనీ మంత్రులకు ఉన్న నోటిదూల ప్రధాన కారణం పేరుకి మాత్రమే మంత్రులు కానీ మీ శాఖల మీద కనీస అవగాహన కూడా ఉండదు చట్టసభలలో ఎలా ప్రవర్తించాలో తెలీదు రేప్ లు జరిగితే అనుకోకుండా చేశారని చెప్తారు మేమేం ఎస్సీ ఎస్టీలు కాదు పక్కన కూర్చోవచ్చు అన్నది నువ్వే కదా రిజల్ట్ ముందు రోజు వరకూ కూడా మీ నోట్లో నుంచి బూతులు తప్ప ఇంకేం వచ్చేవి. ఇప్పుడేమో అమాయకంగా ఎందుకిలా జరిగిందో. నువ్వు ఓడిప

In [50]:
df.shape

(87549, 4)

In [52]:
# Function to generate bigrams
def generate_bigrams(sentence):
    tokens = nltk.word_tokenize(sentence)
    return set(bigrams(tokens))

# Function to compute Jaccard similarity between two sets of bigrams
def jaccard_similarity(bigrams1, bigrams2):
    intersection = len(bigrams1.intersection(bigrams2))
    union = len(bigrams1.union(bigrams2))
    return intersection / union if union != 0 else 0

# Remove near-duplicates based on Jaccard similarity
def remove_near_duplicates(sentences, threshold=0.75):
    unique_sentences = []
    bigrams_list = [generate_bigrams(sentence) for sentence in sentences]
    marked_for_removal = set()

    for i in tqdm(range(len(sentences))):
        if i in marked_for_removal:
            continue
        unique_sentences.append(sentences[i])
        for j in range(i + 1, len(sentences)):
            if j in marked_for_removal:
                continue
            similarity = jaccard_similarity(bigrams_list[i], bigrams_list[j])
            if similarity >= threshold:
                marked_for_removal.add(j)

    return unique_sentences

# Filter the near-duplicate sentences
unique_sentences = remove_near_duplicates(sentences)

# Display the number of unique sentences left
print(f"Number of unique sentences (after removing near duplicates): {len(unique_sentences)}")

100%|██████████████████████████████████████████████████████████████████████████| 87549/87549 [1:22:28<00:00, 17.69it/s]


Number of unique sentences (after removing near duplicates): 86519


In [57]:
df = pd.DataFrame(unique_sentences, columns=["Sentences"])

# Save the DataFrame to a CSV file
df.to_csv("syntactic_filtered_sentences.csv", index=False, encoding='utf-8')